In [1]:
import imp, table_loading
imp.reload(table_loading)

/orange/adamginsburg/miniconda3/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/orange/adamginsburg/miniconda3/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


<module 'table_loading' from '/home/btingle/table_loading.py'>

In [2]:
from table_loading import *

In [3]:
fulltbl = Table.read('/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/SPICY_withAddOns.fits')
fulltbl.add_index('ALMAIMF_FIELDID')
tbl = fulltbl.loc['G328']

read ('/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/SPICY_withAddOns.fits', <_io.FileIO name='/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/SPICY_withAddOns.fits' mode='rb' closefd=True>, <_io.FileIO name='/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/SPICY_withAddOns.fits' mode='rb' closefd=True>) {}


In [4]:
tbl

SPICY,ra,dec,l,b,p1,p2,p3,class,silicate,pah,alpha,alpha_8,alpha_24,alpha_w4,env,group,var,nr,r,sigmar,skewnessr,Spitzer,mag3_6,e_mag3_6,mag4_5,e_mag4_5,mag5_8,e_mag5_8,mag8_0,e_mag8_0,csf,m3_6,m4_5,m5_8,m8_0,2MASS,UKIDSS,VIRAC,GaiaDR2,MIPS,AllWISE,ZTFDR3,in_ALMAIMF,ALMAIMF_FIELDID,70,160,250,350,500,Spitzer/MIPS.24mu_flux,Spitzer/MIPS.24mu_eflux,M24_flux_uplim,RA_ICRS,DE_ICRS,Ksmag,KsEll,pm,e_pm,pmRA,e_pmRA,pmDE,e_pmDE,Nep,flgrel,Zmag,Zell,Ymag,Yell,Jmag,Jell,Hmag,Hell,Zdetflg,Ydetflg,Jdetflg,Hdetflg,ALMA-IMF_3mm_flux,ALMA-IMF_3mm_eflux,ALMA-IMF_1mm_flux,ALMA-IMF_1mm_eflux,Paranal/VISTA.Y_flux,Paranal/VISTA.Y_eflux,Paranal/VISTA.Z_flux,Paranal/VISTA.Z_eflux,Paranal/VISTA.J_flux,Paranal/VISTA.J_eflux,Paranal/VISTA.H_flux,Paranal/VISTA.H_eflux,Paranal/VISTA.Ks_flux,Paranal/VISTA.Ks_eflux,Spitzer/IRAC.I1_flux,Spitzer/IRAC.I1_eflux,Spitzer/IRAC.I2_flux,Spitzer/IRAC.I2_eflux,Spitzer/IRAC.I3_flux,Spitzer/IRAC.I3_eflux,Spitzer/IRAC.I4_flux,Spitzer/IRAC.I4_eflux,Herschel/Pacs.blue_eflux,Herschel/Pacs.red_eflux,Herschel/SPIRE.PSW_eflux,Herschel/SPIRE.PMW_eflux,Herschel/SPIRE.PLW_eflux,Herschel/Pacs.blue_flux,Herschel/Pacs.red_flux,Herschel/SPIRE.PSW_flux,Herschel/SPIRE.PMW_flux,Herschel/SPIRE.PLW_flux
,deg,deg,deg,deg,,,,,,,,,,,,,,,,,,,mag,mag,mag,mag,mag,mag,mag,mag,,,,,,,,,,,,,,,Jy / pix,Jy / pix,MJy / sr,MJy / sr,MJy / sr,mJy,mJy,MJy / sr,deg,deg,mag,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,mag,,mag,,mag,,mag,,,,,,Jy / beam,Jy / beam,Jy / beam,Jy / beam,,,,,,,,,,,mJy,mag mJy,mJy,mag mJy,,mag,,mag,mJy,mJy,mJy,mJy,mJy,,,,,
int64,float64,float64,float64,float64,float32,float32,float32,bytes9,bool,bool,float64,float64,float64,float64,bytes7,bytes10,int16,int16,float64,float64,float64,bytes26,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,bytes18,bytes19,int32,int64,bytes18,bytes20,int64,bool,bytes8,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,int16,uint8,float64,float32,float64,float32,float32,float32,float64,float32,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
31362,239.457389,-53.974193,328.230635,-0.522011,--,--,0.59,uncertain,True,False,-2.04251599326372,-2.04251599326372,--,--,EnvIII,G328.2-0.5,--,--,--,--,--,SSTGLMC G328.2306-00.5220,11.356,0.055,10.367,0.052,9.812,0.053,9.896,0.055,0,2,2,2,2,15574978-5358271,,336878763,--,,,--,True,G328,0.12909437551819858,0.7804596512780834,3451.872583907574,1871.8531236483818,685.8476071588403,--,33.060,34.481247,239.457404,-53.974250,14.703,0.1,0.87,0.96,-0.86,0.96,-0.066,0.964,101,1,--,--,--,--,--,--,--,--,1,1,1,1,--,8.872360412077857e-05,--,0.0003857492389422121,--,--,--,--,--,--,--,--,0.8802201,0.1,7.951118,0.4012032,12.7685585,0.60914224,13.537123,0.6582271,6.8229666,0.34427813,129.09437551819858,780.4596512780834,25047.91972975978,22859.560939461997,16440.980659643323,--,--,--,--,--
31366,239.459999,-53.972003,328.233222,-0.521333,0.71,--,0.59,FS,False,False,-0.11148400244367,0.580368411376471,-0.11148400244367,--,EnvIII,G328.2-0.5,--,--,--,--,--,SSTGLMC G328.2332-00.5213,11.505,0.04,10.921,0.049,10.168,0.071,8.853,0.037,0,2,2,2,2,15575038-5358196,,336878800,5980803429252515072,MG328.2332-00.5213,,--,True,G328,0.1643809998128381,0.9045645176793655,3777.48890459511,1938.0865835816944,759.7098891270746,34.474,1.736,--,239.460020,-53.972072,12.799,0.2,1.75,0.77,1.42,0.77,1.033,0.748,101,1,15.309,0.0611,14.896,0.0639,14.375,0.0477,13.665,0.0993,0,0,0,0,--,8.872360412077857e-05,--,0.0003857492389422121,2.2940891567321575,0.1345,1.703554494247925,0.0955,2.757552,0.1207,3.513271867430251,0.3201,5.083833,0.9,6.93151,0.25436732,7.665532,0.34459728,9.752733,0.6352698,17.830902,0.60526913,164.3809998128381,90

In [5]:
def source_fitting(rownum, geometry="s---s-i",
                   override_valid=None,
                   robitaille_modeldir='/blue/adamginsburg/richardson.t/research/flux/robitaille_models/',):
    
    model_dir = f'{robitaille_modeldir}/{geometry}'
    extinction = make_extinction()
    filters = filternames+["ALMA-IMF_1mm", "ALMA-IMF_3mm"]
    
    apertures = ([3]*(len(filters)))*u.arcsec
    source = Source()
    
    flx, error, valid = get_data_to_fit(rownum, tbl, filters=filters)
    
    if override_valid is not None:
        valid = override_valid

    print(f"flx={flx}")
    print(f"error={error}")
    print(f"valid={valid}")
    
    source.valid = valid
    source.flux = flx 
    source.error = error
    
    fitter = Fitter(filter_names=np.array(filters),
                    apertures=apertures,
                    model_dir=model_dir,
                    extinction_law=extinction,
                    distance_range=[2.3,2.7]*u.kpc,
                    av_range=[5,40],
                   )
    
    fitinfo = fitter.fit(source)
    
    return fitinfo

In [6]:
fitinfosingle = source_fitting(5, "s---s-i")

flx=[           nan            nan            nan            nan
            nan 4.89359558e-01 1.07979715e+00 2.26321840e+00
 1.98597252e+00 7.56676666e+02 1.15822742e+03 4.92234693e+03
 1.26091914e+05 1.16612614e+05 9.30056486e+04 1.35572657e-03
 1.37206291e-03]
error=[           nan            nan            nan            nan
            nan 5.25275804e-02 1.19867392e-01 2.88612247e-01
 2.14995176e-01 9.97000000e-01 9.97000000e-01 9.97000000e-01
 9.97000000e-01 9.97000000e-01 9.97000000e-01 3.85749239e-04
 8.87236041e-05]
valid=[0 0 0 0 0 1 1 1 1 3 3 3 3 3 3 1 1]
 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  s---s-i
   Log[d] stepping     :  0.02
   Number of distances :  5

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /blue/adamginsbu

   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Spitzer/IRAC.I3.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Spitzer/IRAC.I4.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Spitzer/MIPS.24mu.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Herschel/Pacs.blue.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Herschel/Pacs.red.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Herschel/SPIRE.PSW.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Herschel/SPIRE.PMW.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/convolved/Herschel/SPIRE.PLW.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/robitai

In [7]:
# check if things are still broke
print(fitinfosingle.av)
print(np.isfinite(fitinfosingle.av).sum())
print(fitinfosingle.chi2)
print(np.isfinite(fitinfosingle.chi2).sum())

[nan nan nan ... nan nan nan]
0
[nan nan nan ... nan nan nan]
0


In [7]:
fitinfosingle = source_fitting(1, "s---s-i", override_valid=[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3])

flx=[2.29408916e+00 1.70355449e+00 2.75755191e+00 3.51327187e+00
 5.08383322e+00 6.93150997e+00 7.66553211e+00 9.75273323e+00
 1.78309021e+01 3.44740000e+01 4.93142999e+02 2.71369355e+03
 8.22320957e+04 7.10052639e+04 5.46347707e+04 1.15724772e-03
 2.66170812e-04]
error=[0.13448835 0.09549282 0.12067451 0.32006226 0.93281341 0.25436732
 0.34459728 0.63526982 0.60526913 1.73599994 0.997      0.997
 0.997      0.997      0.997      0.997      0.997     ]
valid=[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3]
 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  s---s-i
   Log[d] stepping     :  0.02
   Number of distances :  5

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/co

In [8]:
# check if things are still broke
print(fitinfosingle.av)
print(np.isfinite(fitinfosingle.av).sum())
print(fitinfosingle.chi2)
print(np.isfinite(fitinfosingle.chi2).sum())

[5.99873668 6.2251738  5.30560538 ... 5.         5.         5.        ]
10000
[  6686.66468839   6776.51601857   6818.73797547 ... 495428.05126263
 501407.03636251 509511.09224826]
10000


In [9]:
fitinfosingle = source_fitting(1, "s---s-i", override_valid=[1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3])

flx=[2.29408916e+00 1.70355449e+00 2.75755191e+00 3.51327187e+00
 5.08383322e+00 6.93150997e+00 7.66553211e+00 9.75273323e+00
 1.78309021e+01 3.44740000e+01 4.93142999e+02 2.71369355e+03
 8.22320957e+04 7.10052639e+04 5.46347707e+04 1.15724772e-03
 2.66170812e-04]
error=[0.13448835 0.09549282 0.12067451 0.32006226 0.93281341 0.25436732
 0.34459728 0.63526982 0.60526913 1.73599994 0.997      0.997
 0.997      0.997      0.997      0.997      0.997     ]
valid=[1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3]
 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  s---s-i
   Log[d] stepping     :  0.02
   Number of distances :  5

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /blue/adamginsburg/richardson.t/research/flux/robitaille_models//s---s-i/co

In [10]:
# check if things are still broke
print(fitinfosingle.av)
print(np.isfinite(fitinfosingle.av).sum())
print(fitinfosingle.chi2)
print(np.isfinite(fitinfosingle.chi2).sum())

[5.99873668 6.2251738  5.30560538 ... 5.         5.         5.        ]
10000
[  6692.06525149   6781.91658166   6824.13853856 ... 495428.05126263
 501407.03636251 509511.09224826]
10000
